# Capstone Project - The Battle of the Neighrborhoods 

### Applied Data Science by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this experiment called 'The Battle of Neighbourhoos' I am going to show how similar and dissimilar two random places in New York and Toronto can be and which one is the best place to start a business. First, in New York there are 5 boroughs and I decided to work with Brooklyn. On the other hand, Toronto has 6 districs and I chose North York.

Brooklyn and North York are very similar cities in overall perpesctives, that's why I decided to work with them. However, Brooklyn has many more neighbourhoods althoguh the difference between their areas,which is just a little bit(we can omit it). Therefore, the neighbourhoods of North York are larger.

This report is specially intended for people who want to start a business. There is a lot of fields where you can start something, but there is not so much space and this countries are first world, so it is difficult to get a space and paying a rent could be very expensive for most people. As immigration in these countries is not pretty difficult, people from poor countries can get good jobs and then start their own business.


## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing venues within each neighborhood.
* number of existing venues within each neighborhood belong to a category.
* 10 most common venues with each neighborhood

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**

### Data of New York

In [1]:
# Import the required libraries
import numpy as np
import pandas as pd
import json
import requests
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import wget
CLIENT_ID = 'NY40LS44LI5LOLQLNG4JATGX55KPAO212YRFAJ1Z52MA3M1U'
CLIENT_SECRET = 'ERAVXBOIN4X54NKVHRCIMR02P1F033XT45XFJI5ZL3IFT0OF'
ACCESS_TOKEN = '5AXUAIRZSQSCYNYSZ44JC0PMBAJLBPGF1F1QZYE3QEXCBEO4'
VERSION = '20180604'

In [2]:
# url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json'
# filename = wget.download(url)
# filename
# Like I already have the data only I need to call.

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [3]:
# Create our labels on the data
column_names =  ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
new_york = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    new_york = new_york.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
new_york.head()  

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [4]:
# Get the location of New York City
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location_NY = geolocator.geocode(address)
latitude_NY = location_NY.latitude
longitude_NY = location_NY.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude_NY, longitude_NY))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### Data of Toronto

In [5]:
# Like I already have the data, only I need to call by pandas
df_1T = pd.read_excel ('D:\python\data analysis\Data_Toronto.xlsx')
df_1T.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
# Use the function drop to eliminate all values 'Not assigned' 
for i in range(0,df_1T.shape[0]):
    if df_1T.loc[i,'Borough']== 'Not assigned':
        df_1T=df_1T.drop(i)
        df_1T.reset_index(drop=True)
df_1T.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
# Download location Data https://cocl.us/Geospatial_data
df_location_1T = pd.read_csv('D:\python\data analysis\Geospatial_Coordinates.csv')
df_location_1T.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
# Merge our two datas into one on Postal Code like reference
toronto = pd.merge(df_1T, df_location_1T,how='left',on='Postal Code')
toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [9]:
# Get the location of Toronto
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location_T = geolocator.geocode(address)
latitude_T = location_T.latitude
longitude_T = location_T.longitude
print('The geograpical coordinate of Toronto {}, {}.'.format(latitude_T, longitude_T))

The geograpical coordinate of Toronto 43.6534817, -79.3839347.


### Data specifically of Brooklyn

In [10]:
# Get the data of Brooklyn
brooklyn_data = new_york[new_york['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [11]:
# Get the location of Brooklyn
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location_bro = geolocator.geocode(address)
latitude_bro = location_bro.latitude
longitude_bro = location_bro.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude_bro, longitude_bro))

# Get a map of venues in Brooklyn
map_bro = folium.Map(location=[latitude_bro, longitude_bro], zoom_start=11)

for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bro)  
    
map_bro

The geograpical coordinate of Manhattan are 40.6501038, -73.9495823.


In [12]:
# Get data of Gravesend
brooklyn_data.loc[4, 'Neighborhood']

'Gravesend'

In [13]:
# Get location of Gravesend
gravesend_latitude = brooklyn_data.loc[4, 'Latitude'] 
gravesend_longitud = brooklyn_data.loc[4, 'Longitude']

gravesend_name = brooklyn_data.loc[4, 'Neighborhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(gravesend_name, 
                                                               gravesend_latitude, 
                                                               gravesend_longitud))

Latitude and longitude values of Gravesend are 40.59526001306593, -73.97347087708445.


In [14]:
# Get API
LIMIT = 100 
radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    gravesend_latitude, 
    gravesend_longitud, 
    radius, 
    LIMIT)

results1 = requests.get(url).json()

In [15]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
# Get near venues to Brooklyn
venuesB = results1['response']['groups'][0]['items']
nearby_venuesB = pd.json_normalize(venuesB) 
filtered_columnsB = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venuesB =nearby_venuesB.loc[:, filtered_columnsB]
nearby_venuesB['venue.categories'] = nearby_venuesB.apply(get_category_type, axis=1)
nearby_venuesB.columns = [col.split(".")[-1] for col in nearby_venuesB.columns]

nearby_venuesB.head()

,name,categories,lat,lng
0,Joe's of Avenue U,Italian Restaurant,40.596971,-73.973677
1,Nuccios Bakery,Bakery,40.597097,-73.974483
2,Living Room Cafe,Lounge,40.596439,-73.977583
3,Dunkin',Donut Shop,40.596786,-73.976686
4,Elegante Pastry Shop,Bakery,40.596633,-73.978182


In [17]:
print('{} venues were returned by Foursquare.'.format(nearby_venuesB.shape[0]))

31 venues were returned by Foursquare.


### Explore Neighborhoods in Brooklyn

In [18]:
# Get data of nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
# Get neighbourhoods within North York
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude'])

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [20]:
# Show location of neighbourhoods
print(brooklyn_venues.shape)
brooklyn_venues.head()

(2754, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Georgian Dream Cafe and Bakery,40.625586,-74.030196,Caucasian Restaurant
3,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.624200,-74.030931,Pizza Place
4,Bay Ridge,40.625801,-74.030621,Cocoa Grinder,40.623967,-74.030863,Juice Bar


In [21]:
# Get the total of venues in Brooklyn by neighbourhood
brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,47,47,47,47,47,47
Bay Ridge,84,84,84,84,84,84
Bedford Stuyvesant,31,31,31,31,31,31
Bensonhurst,33,33,33,33,33,33
Bergen Beach,6,6,6,6,6,6
...,...,...,...,...,...,...
Vinegar Hill,28,28,28,28,28,28
Weeksville,15,15,15,15,15,15
Williamsburg,35,35,35,35,35,35


In [22]:
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]
brooklyn_onehot.head()

,Yoga Studio,Accessories Store,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,...,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yemeni Restaurant
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
brooklyn_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,...,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yemeni Restaurant
0,Bath Beach,0.0,0.021277,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.021277,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,Bay Ridge,0.0,0.000000,0.0,0.0,0.035714,0.0,0.0,0.0,0.011905,...,0.011905,0.0,0.011905,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2,Bedford Stuyvesant,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.032258,0.032258,0.0,0.0,0.0
3,Bensonhurst,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
4,Bergen Beach,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


In [24]:
# Show the 10 most common venues within each neighbourhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

brooklyn_venues_sorted = pd.DataFrame(columns=columns)
brooklyn_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    brooklyn_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

brooklyn_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Chinese Restaurant,Sushi Restaurant,Fast Food Restaurant,Gas Station,Donut Shop,Bubble Tea Shop,Italian Restaurant,Pharmacy,Cosmetics Shop,Clothing Store
1,Bay Ridge,Spa,Pizza Place,Italian Restaurant,Greek Restaurant,Grocery Store,American Restaurant,Bar,Bagel Shop,Café,Playground
2,Bedford Stuyvesant,Coffee Shop,Deli / Bodega,Bar,Café,Pizza Place,Japanese Restaurant,New American Restaurant,Gift Shop,Gourmet Shop,Boutique
3,Bensonhurst,Pizza Place,Chinese Restaurant,Donut Shop,Grocery Store,Sushi Restaurant,Ice Cream Shop,Italian Restaurant,Cha Chaan Teng,Supermarket,Smoke Shop
4,Bergen Beach,Harbor / Marina,Baseball Field,Playground,Athletics & Sports,Park,Non-Profit,Noodle House,Nightclub,North Indian Restaurant,Opera House
5,Boerum Hill,Coffee Shop,Bar,Dance Studio,French Restaurant,Furniture / Home Store,Bakery,Arts & Crafts Store,Sandwich Place,Grocery Store,Spa
6,Borough Park,Bank,Pizza Place,Fast Food Restaurant,Pharmacy,Metro Station,Chinese Restaurant,American Restaurant,Bistro,Deli / Bodega,Restaurant
7,Brighton Beach,Russian Restaurant,Eastern European Restaurant,Restaurant,Gourmet Shop,Sushi Restaurant,Pharmacy,Bank,Mobile Phone Shop,Beach,Supplement Shop
8,Broadway Junction,Gas Station,Donut Shop,Diner,Sandwich Place,Fried Chicken Joint,Burger Joint,Nightclub,Breakfast Spot,Caribbean Restaurant,Mexican Restaurant
9,Brooklyn Heights,Deli / Bodega,Yoga Studio,Italian Restaurant,Pharmacy,Gym,Coffee Shop,Park,Pizza Place,Cosmetics Shop,Pet Store


### Data specifically of North York

In [25]:
# get the data of North York
NorthYork_data = toronto[toronto['Borough'] == 'North York'].reset_index(drop=True)
NorthYork_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [26]:
# Get the location of North York
address = 'North York'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

# Get a map of venues in North York
NorthYork_map = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(NorthYork_data['Latitude'], NorthYork_data['Longitude'], NorthYork_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(NorthYork_map)  
    
NorthYork_map

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [27]:
# Get a data of Glencairn
NorthYork_data.loc[4, 'Neighbourhood']
# Get the location of Glencairn
Glencairn_latitude = NorthYork_data.loc[4, 'Latitude'] 
Glencairn_longitude = NorthYork_data.loc[4, 'Longitude']

Glencairn_name = NorthYork_data.loc[4, 'Neighbourhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(Glencairn_name, 
                                                               Glencairn_latitude, 
                                                               Glencairn_longitude))

Latitude and longitude values of Glencairn are 43.709577, -79.4450726.


In [28]:
# Get API
LIMIT = 100 
radius = 500 
url_NoY = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Glencairn_latitude, 
    Glencairn_longitude, 
    radius, 
    LIMIT)

results_NoY = requests.get(url_NoY).json()

In [29]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
# Get near venues to North York
venues = results_NoY['response']['groups'][0]['items']
nearby_venues = pd.json_normalize(venues) 
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,"R Bakery - Delicious Cakes, Breads",Bakery,43.707420,-79.443126
1,Miyako Sushi Restaurant,Japanese Restaurant,43.709111,-79.443930
2,Li Cheng Restaurant,Asian Restaurant,43.708828,-79.443366
3,Glencairn Subway Station,Metro Station,43.708872,-79.440801
4,Fraserwood Park,Park,43.713550,-79.442482


In [32]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


### Explore Neighborhoods in North York

In [33]:
# Get data of nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
#Get the neighbourhoods in North York
NorthYork_venues = getNearbyVenues(names=NorthYork_data['Neighbourhood'],
                                   latitudes=NorthYork_data['Latitude'],
                                   longitudes=NorthYork_data['Longitude'])

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [35]:
# Get location and category of venues
print(NorthYork_venues.shape)
NorthYork_venues.head()

(246, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
3,Parkwoods,43.753259,-79.329656,GreenWin pool,43.756232,-79.333842,Pool
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [36]:
# Get the total of venues in Brooklyn by neighbourhood
NorthYork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
Don Mills,26,26,26,26,26,26
Downsview,14,14,14,14,14,14
"Fairview, Henry Farm, Oriole",64,64,64,64,64,64
Glencairn,5,5,5,5,5,5
Hillcrest Village,6,6,6,6,6,6
Humber Summit,2,2,2,2,2,2


In [37]:
# Show the number of venues in Brooklyn
NorthYork_onehot = pd.get_dummies(NorthYork_venues[['Venue Category']], prefix="", prefix_sep="")

NorthYork_onehot['Neighborhood'] = NorthYork_venues['Neighborhood'] 

fixed_columns = [NorthYork_onehot.columns[-1]] + list(NorthYork_onehot.columns[:-1])
NorthYork_onehot = NorthYork_onehot[fixed_columns]

NorthYork_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Supermarket,Supplement Shop,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
# Show the proportion, within North York, of venues
NorthYork_grouped = NorthYork_onehot.groupby('Neighborhood').mean().reset_index()
NorthYork_grouped.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Supermarket,Supplement Shop,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.095238,...,0.047619,0.0,0.047619,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
1,Bayview Village,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.250000,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
2,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.038462,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.038462,0.038462,0.0,0.038462,0.0,0.0,0.0,0.0
3,Don Mills,0.0,0.000000,0.000000,0.038462,0.0,0.038462,0.038462,0.0,0.000000,...,0.038462,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
4,Downsview,0.0,0.071429,0.000000,0.000000,0.0,0.000000,0.071429,0.0,0.071429,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [39]:
# Show the 10 most common venues within each neighbourhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = NorthYork_grouped['Neighborhood']

for ind in np.arange(NorthYork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NorthYork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Fried Chicken Joint,Mobile Phone Shop,Diner,Park,Pizza Place,Restaurant,Middle Eastern Restaurant
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Accessories Store,Luggage Store,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant
2,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Pizza Place,Cupcake Shop,Pub,Comfort Food Restaurant,Ice Cream Shop,Pharmacy,Fast Food Restaurant
3,Don Mills,Gym,Restaurant,Coffee Shop,Café,Supermarket,Japanese Restaurant,Grocery Store,Discount Store,Dim Sum Restaurant,Clothing Store
4,Downsview,Park,Bank,Coffee Shop,Electronics Store,Food Truck,Shopping Mall,Baseball Field,Grocery Store,Hotel,Airport
5,"Fairview, Henry Farm, Oriole",Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Japanese Restaurant,Cosmetics Shop,Chocolate Shop,Juice Bar,Food Court,Jewelry Store
6,Glencairn,Park,Asian Restaurant,Bakery,Metro Station,Japanese Restaurant,Lounge,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant
7,Hillcrest Village,Fast Food Restaurant,Golf Course,Athletics & Sports,Pool,Mediterranean Restaurant,Dog Run,Liquor Store,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station
8,Humber Summit,Pizza Place,Intersection,Accessories Store,Liquor Store,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station,Men's Store,Mediterranean Restaurant
9,"Humberlea, Emery",Furniture / Home Store,Baseball Field,Accessories Store,Liquor Store,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station,Men's Store,Mediterranean Restaurant


## Methodology <a name="methodology"></a>

In this project we will focus on three main graphs for every target place(Brooklyn and New York), we wil limite our analysis within 500m radio. 

At first, We already have the data required: number of venues within every neighbourhood and with their respective category, a map representing the co-relacion with their respective space on the ground, and the 10 most common venues within each neighborhood.

Second step in our analysis will be a brief investigacion about the geography of every borough in order to see the substantial differences between them, making use of our map.

In third and final step we will combine the 2 other graphs we have in order to see the preferencies in every city and find suitable places for starting a business on your own, so you can be confident that you are taking a good decision.

## Analysis <a name="analysis"></a>

###  Let's see our two maps

In [44]:
NorthYork_map

In [45]:
map_bro

##### Main Point to keep in mind about every town.
Brooklyn
* It is almost totally surrounded by the sea.
* Venues are randomly located.
* Near to the sides has brief rivers and green space, but within the town there isn't almost anything.
* It has a great platform for the ships.

North York
* Venues seem to have a pattern behind their position on the ground.
* It has its own airport.
* It has great green spaces and rivers.

About both
* Brooklyn has more venues than North York.
* They have almost the same ground.
* Brooklyn has more neighborhoods.

Based on our deductions from the maps we can say that in Brooklyn you can find more easily jobs, because there are more venues. There are aslo more field in which you can find or start a new business becasue there is the sea near to you. Similar to the last decution we can say that in Brooklyn, due to the platfor for ships, it could have more opportunities to work in that field. Due to the random location of venues you can find out a strange but good location to start a business. 

Instead, In North York you have mainly an airport, it means more tourist and easy sendings. It's a good option due to the green spaces as well. But due to the pattern which follow the venues it wouls be difficult find a suitable space to start a business, however you have more space and less neighborhoods, so you can invest in the correct space for the correct category.

In contrast, Brooklyn has more atractive and touristy places, and North York has more space and field to invest in. Brooklyn has more choices like fishing, while North York has its own aiport which can ease the transit of goods and people.

###  Let's see the numebr of venues in every place.

In [49]:
brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,47,47,47,47,47,47
Bay Ridge,84,84,84,84,84,84
Bedford Stuyvesant,31,31,31,31,31,31
Bensonhurst,33,33,33,33,33,33
Bergen Beach,6,6,6,6,6,6
...,...,...,...,...,...,...
Vinegar Hill,28,28,28,28,28,28
Weeksville,15,15,15,15,15,15
Williamsburg,35,35,35,35,35,35


In [50]:
NorthYork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
Don Mills,26,26,26,26,26,26
Downsview,14,14,14,14,14,14
"Fairview, Henry Farm, Oriole",64,64,64,64,64,64
Glencairn,5,5,5,5,5,5
Hillcrest Village,6,6,6,6,6,6
Humber Summit,2,2,2,2,2,2


Again you can see the great difference in the number of venues between them. Even in North York there is two neighborhoods where only is considered 1 venue like open. Whilst the minimum in Brooklyn is 22. This is a interesting point to address because knowing that Brooklyn has more neighborhoods and venues than North York and with the same ground, we can infer that Brooklyn is a city more active and North York has pretty many options where you can start a business with the certeinty that it will worth. When we talk about option not only we cover the field you can choose for starting a business, the cities where you can do something as weel. In Brooklyn would be finding a job easier but not on your own, you can find for sure many more companies and enterprises which have already started. 

You can understand this saying that if you prefer something pretty straighforward you can choose finding a job in Brooklyn, but obviously you will depend on your background and for sure you will have a certain wage which difficulty it will can raise. Otherwise In North York the main point in order to generate profit would be starting a new business from cero, I consider this more hard to complete because most people have never leaded with this in their lives; but you have the option to grow up fairly fast thanks to the poor in this last, according to these graphs.

###  Let's see the ten most common venues in every neighborhood.

In [51]:
brooklyn_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Chinese Restaurant,Sushi Restaurant,Fast Food Restaurant,Gas Station,Donut Shop,Bubble Tea Shop,Italian Restaurant,Pharmacy,Cosmetics Shop,Clothing Store
1,Bay Ridge,Spa,Pizza Place,Italian Restaurant,Greek Restaurant,Grocery Store,American Restaurant,Bar,Bagel Shop,Café,Playground
2,Bedford Stuyvesant,Coffee Shop,Deli / Bodega,Bar,Café,Pizza Place,Japanese Restaurant,New American Restaurant,Gift Shop,Gourmet Shop,Boutique
3,Bensonhurst,Pizza Place,Chinese Restaurant,Donut Shop,Grocery Store,Sushi Restaurant,Ice Cream Shop,Italian Restaurant,Cha Chaan Teng,Supermarket,Smoke Shop
4,Bergen Beach,Harbor / Marina,Baseball Field,Playground,Athletics & Sports,Park,Non-Profit,Noodle House,Nightclub,North Indian Restaurant,Opera House
5,Boerum Hill,Coffee Shop,Bar,Dance Studio,French Restaurant,Furniture / Home Store,Bakery,Arts & Crafts Store,Sandwich Place,Grocery Store,Spa
6,Borough Park,Bank,Pizza Place,Fast Food Restaurant,Pharmacy,Metro Station,Chinese Restaurant,American Restaurant,Bistro,Deli / Bodega,Restaurant
7,Brighton Beach,Russian Restaurant,Eastern European Restaurant,Restaurant,Gourmet Shop,Sushi Restaurant,Pharmacy,Bank,Mobile Phone Shop,Beach,Supplement Shop
8,Broadway Junction,Gas Station,Donut Shop,Diner,Sandwich Place,Fried Chicken Joint,Burger Joint,Nightclub,Breakfast Spot,Caribbean Restaurant,Mexican Restaurant
9,Brooklyn Heights,Deli / Bodega,Yoga Studio,Italian Restaurant,Pharmacy,Gym,Coffee Shop,Park,Pizza Place,Cosmetics Shop,Pet Store


In [52]:
neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Fried Chicken Joint,Mobile Phone Shop,Diner,Park,Pizza Place,Restaurant,Middle Eastern Restaurant
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Accessories Store,Luggage Store,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant
2,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Pizza Place,Cupcake Shop,Pub,Comfort Food Restaurant,Ice Cream Shop,Pharmacy,Fast Food Restaurant
3,Don Mills,Gym,Restaurant,Coffee Shop,Café,Supermarket,Japanese Restaurant,Grocery Store,Discount Store,Dim Sum Restaurant,Clothing Store
4,Downsview,Park,Bank,Coffee Shop,Electronics Store,Food Truck,Shopping Mall,Baseball Field,Grocery Store,Hotel,Airport
5,"Fairview, Henry Farm, Oriole",Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Japanese Restaurant,Cosmetics Shop,Chocolate Shop,Juice Bar,Food Court,Jewelry Store
6,Glencairn,Park,Asian Restaurant,Bakery,Metro Station,Japanese Restaurant,Lounge,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant
7,Hillcrest Village,Fast Food Restaurant,Golf Course,Athletics & Sports,Pool,Mediterranean Restaurant,Dog Run,Liquor Store,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station
8,Humber Summit,Pizza Place,Intersection,Accessories Store,Liquor Store,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station,Men's Store,Mediterranean Restaurant
9,"Humberlea, Emery",Furniture / Home Store,Baseball Field,Accessories Store,Liquor Store,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station,Men's Store,Mediterranean Restaurant


This work like a sample where you can see several similarities and differences within the frame. Although both stand out in venues for food there is a small difference in North York, while in Brooklyn people prefer to eat food from different places around the current world, people in North York prefer to eat food from different ages, for instance there are 6 Middle Eastern restaurants while there is no thing in Brooklyns, from the same way in Brooklyn there are 4 italian restaurants whereas there is only one in North York.

Another curiosity is leisure is more development in Brooklyn, you can find cosmetic shops ans spas while in NY there is not. You have to remember that Brooklyn has a great beach, therefore there is more leisure. So a good option would be start a business in Brooklyn pretty near to leisure, like enhace it, or search more variety to people who live in North York, instead Mediterranean Food it would be well to create centers and platform to learn more about Mediterranean and its culture, obviously including food and how to prepare.

## Results and Discussion <a name="results"></a>

Our analysis show the advanteges and disadvanteges in every city start a new job. Brooklyn is almost totally surrounded by the sea, which generate a great interest in fiels related to it, like beach; its venues are randomly located, which generate some suitable spaces to start a business; and by last it has a platform for the ship, so you can think, besides fishing, in means of transport and investigation of the sea. While North York seems to have a pattern with regard to location of venues, which generate some difficulties to find a good space to put a enterprise without too risk; it aslo has its own airport, which ease the transport with other cities; and by last it has great spaces and rivers, this is maybe why there is not many space to neighborhood and venues neither.

I want also highlight the great difference between the number of venues in every neighborhood, because in spite of outstanding position of venues in North York, you could find out a good space for your company, becase this place lacks venues or at least big centers. A good idea would be invest in one of the two neighborhoods you saw in the frame, which have only 1 remarcable venue in order to try getting a better posibility to growp up easier . But we also can think in the option to work in a company to then start your own enterprise, for that purpose the best option between these two place would be Brooklyn.

Results of the most common venues provide us the option to see what people prefer and some especial characteristics. We remarked that people prefer food from different countries around the world in Brooklyn, while in North York people prefer food from different ages; as well as in Brooklyn there is more leisure and pleople interested in that.

In regard to our data we find Brooklyn like a city pretty more interesting than North York; because has a beach, a  platform for ships, and only you need your imagination to find out a gripping field where you can invest without dependinig on fortune. However, altough North York does not have several fields to invest in, it has more space and less venues recognized, so you have more options at least.

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify advanteges and disadvanteges between Brooklyn and North York(,like a reference of therir repestive countries) in order to find the best option to invest and start and new business. By examining deeply main characteristics within every neighborhood, and then see the ten most common venues like one else reference in order to see specifically the preferences and subtle differences people belong to every borough.

Final desicion on best borough to start a business is Brooklyn, based on that it has a great beach, a platform for the ships and large spaces specifically dedicated to leisure. People have what they want, but you can create something new and benefit from all the things that are already there, you can lean on them and get to achieve many more things, because you already have a base.